In [1]:
import os, sys
import time
import numpy as np
import pandas as pd
import random
from scipy import stats as st
import itertools
import operator
import heapq as hq
import torch

from tqdm.notebook import trange
from tqdm import tqdm

# Init steps

In [2]:
# get currently working directory
base_dir = os.getcwd()

# load functions from other notebooks
helpers_file = os.path.join(base_dir, 'helpers.ipynb').replace("\\", "/")
%run $helpers_file

In [3]:
for p in ['../spotlight_ext']:
    module_path = os.path.abspath(os.path.join(base_dir, p))
    if module_path not in sys.path:
        sys.path.append(module_path)

random_state = np.random.RandomState(2020)

# Prepare models/datasets
#### Load the pretrained models "lstm" (entire_model_1m_20interactions.pt) and "pooling" (pooling_model_1m_20interactions.pt)

In [4]:
# implicit_model = load_model('implicit_factorization')
lstm_model = load_model(model_type='entire')
pooling_model = load_model('pooling')

pretrained_models = {
    'lstm': lstm_model,
    'pooling': pooling_model,
}

#### Get the dataset Movielens with the variant 1M. Then divide it into a training set and a testing set. And finally this code limits the length of each sequence of elements in the 2 sets to 20.

In [5]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset

# get dataset 
dataset = get_movielens_dataset(variant='1M')
train, test = random_train_test_split(dataset, random_state=random_state)

max_sequence_length = 20
train = train.to_sequence(max_sequence_length=max_sequence_length)
test = test.to_sequence(max_sequence_length=max_sequence_length)

#### 1. Compute the similarity matrix based on cosine similarity
#### 2. Compute the similarity matrix based on Jaccard similarity

In [6]:
pooling_sims_matrix = gpu_embeddings_to_cosine_similarity_matrix(
    pooling_model._net.item_embeddings(
        torch.arange(0, dataset.num_items, dtype=torch.int64)
    )).detach().numpy()

jaccard_sims_matrix = compute_sim_matrix(dataset, 'jaccard')

  0%|          | 0/6040 [00:00<?, ?it/s]

# Various implemented Strategies

In [7]:
class BaseStrategy:
    class_name = None

    def __init__(self, item, interactions, max_length, init_budget,  model=None, random_pick=False):

        self.target_item = item
        self.original_interactions = interactions
        self.max_length = max_length
        self.visited_ = set()
        self.model = model
        self.last_comb_cost = 0
        self.random_pick = random_pick
        self.top_k = 10
        self.budget = init_budget

    #Must be implemented by subclasses. Used to select the next item to recommand to the user.
    def next_comb(self, reverse=False):
        raise NotImplementedError

    
    def _get_pos(self, number):
        bits = []
        for i, c in enumerate(bin(number)[:1:-1], 1):
            if c == '0':
                bits.append(i)
        return bits

    #Method to reset the costs of the last recommended combination
    def reset_costs(self):
        self.last_comb_cost = 0

    #Returns the initial budget
    def get_init_budget(self):
        return self.budget

### RandomSelection
#### This class is a subclass of the "BaseStrategy" class, representing a random item selection strategy for the sequential recommendation task.

In [8]:
class RandomSelection(BaseStrategy):
    class_name = 'Random'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model):
        super().__init__(item, interactions, max_sequence_length, init_budget)

    # The _next_item method selects a random integer between 1 and the maximum length of the sequence. 
    # It checks if the integer is already in the set of visited integers, and if it is, selects another integer until a non-visited integer is found. 
    # Finally, the selected integer is added to the set of visited integers, and returned.
    def _next_item(self):
        self.budget -= 1
        
        number = random.sample(range(1, pow(2, self.max_length)), 1)[0]
        while number in self.visited_:
            number = random.sample(range(1, pow(2, self.max_length)), 1)[0]
        self.visited_.add(number)
        return number
    
    #The next_comb method generates a new sequence by removing items at positions indicated by the binary digits of the integer returned 
    # by _next_item from the original sequence of interactions. 
    # The resulting sequence, along with the current budget, is returned as a tuple.
    def next_comb(self, reverse=False):
        number = self._next_item()

        bits = self._get_pos(number)
        seq = np.delete(self.original_interactions, bits)

        return (seq, self.budget)

### LossSimilarSelection
#### This class inherits frome the BaseStrategy class. This class defines a search strategy for selecting items based on their similarity to previously selected items, while also considering their loss (difference between predicted and actual values) in a ranking problem.

In [9]:
class LossSimilarSelection(BaseStrategy):
    class_name = 'BFS'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model, early_term=False):
        
        
        super().__init__(item, interactions, max_sequence_length, init_budget, model)

        self.q = Queue()
        self.q.enqueue(([False] * len(self.original_interactions), StaticVars.INT_MAX, 0))

        self.thres = len(self.original_interactions) + 1
        self.early_termination = early_term

    # Helper : This method is called whenever a solution is found for the current 
    # mask of the items. It computes the loss of the solution and updates the queue 
    # accordingly.
    def _update_queue(self, is_solved):
        self.compute_loss(is_solved)

    # Helper : 
    def _next_item(self):
        mask, t_score, is_solved = self.q.dequeue()
        while self.early_termination and sum(mask) == self.thres:
            q_data = self.q.dequeue()
            if q_data is None: break

            mask, t_score, is_solved = q_data

        if is_solved == 2:
            t_score, kth_score = self.get_score(mask)

            if (t_score / kth_score) < 1: self.thres = sum(mask)

        return (is_solved, mask, self.budget)

    # The next_comb method returns the next combination of items and the remaining budget.
    def next_comb(self, reverse=False):
        budget = self.budget

        if self.q.size() > 0:
            solved_flag, item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())
            self._update_queue(solved_flag) #bug here
        else: self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr)
        return (seq, budget) if len(seq) else (None, budget)

    # This method computes the loss of the solution. If the solution is not yet solved, it searches for the next combination of items 
    # to evaluate by calling the search method. 
    # If the solution is solved, it searches for the previous combination of items by calling the search method with forward=False.
    def compute_loss(self, is_solved=False):
        self.last_comb_cost = 0

        if not is_solved: self.search(forward=True, s=is_solved)
        else: self.search(forward=False, s=is_solved)

    def search(self, forward=True, s=False):
        m_mask = np.ma.getmask(self.ma_arr).copy()
        valid_items = np.where(np.logical_not(m_mask) if forward else m_mask)[0]
        if valid_items.size > 1:
            for idx in valid_items:
                m_mask[idx] = not m_mask[idx]
                self.add(m_mask, s)
                m_mask[idx] = not m_mask[idx]

    # The get_score method returns the score of an item based on its predicted value and its rank among the top-k items.
    def get_score(self, d):
        perm = np.ma.compressed(np.ma.masked_array(self.original_interactions, mask=d))

        self.budget -= 1
        # predict next top-k items about to be selected
        preds = self.model.predict(perm)
        preds[perm] = -StaticVars.FLOAT_MAX
        rk_data = st.rankdata(-preds, method='ordinal')

        return (preds[self.target_item], preds[(rk_data == self.top_k).nonzero()][0])

    # The add method adds a new combination to the queue and updates the visited set.
    def add(self, d, s):
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if (mask_to_int not in self.visited_) and (self.budget > 0):
            perm = np.ma.compressed(np.ma.masked_array(self.original_interactions, mask=d))

            if not s:
                t_score, kth_score = self.get_score(d)

                if self.q.size() == 0: self.q.enqueue((d.copy(), t_score, 1 if (t_score / kth_score) < 1 else 0))

                if t_score < self.q.get(0)[1]:  # get only the assigned score
                    self.q.setter(0, (d.copy(), t_score, 1 if (t_score / kth_score) < 1 else 0))
            else:
                self.q.enqueue((d.copy(), StaticVars.INT_MAX, 2))

            self.visited_.add(mask_to_int)

### BiDirectionalSelection

In [10]:
class BiDirectionalSelection(BaseStrategy):
    class_name = 'BiDirectional'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model, weights=(1, 0), alpha=0.9, normalization='default'):
        super().__init__(item, interactions, max_sequence_length, init_budget, model)

        self.tiebraker = itertools.count()
        self.q = [(1, StaticVars.INT_MAX, next(self.tiebraker), [False] * len(self.original_interactions), self.budget)]
        hq.heapify(self.q)

        self.alpha = alpha
        self.norm = normalization

    def _update_queue(self, is_solved):
        self.compute_loss(is_solved)

    def _next_item(self):
        is_solved, _, _, mask, budget = hq.heappop(self.q)
        return (is_solved, mask, budget)

    def next_comb(self, reverse=False):
        budget = self.budget
        if self.q:
            solved_flag, item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())
            self._update_queue(solved_flag)
        else: self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr)
        return (seq, budget) if len(seq) else (None, budget)

    def compute_loss(self, is_solved=False):
        self.search(forward=True, s=is_solved)
        self.search(forward=False, s=is_solved)

    def search(self, forward=True, s=False):
        m_mask = np.ma.getmask(self.ma_arr).copy()
        valid_items = np.where(np.logical_not(m_mask) if forward else m_mask)[0]
        if valid_items.size > 1:
            for idx in valid_items:
                m_mask[idx] = not m_mask[idx]
                self.add(m_mask, s)
                m_mask[idx] = not m_mask[idx]

    def get_custom_score(self, c):
        return c / self.max_length

    def get_score(self, d):
        self.budget -= 1

        # predict next top-k items about to be selected
        perm = np.ma.compressed(np.ma.masked_array(self.original_interactions, mask=d))
        preds = self.model.predict(perm)

        if self.norm == 'kth_norm':
            preds[perm] = -StaticVars.FLOAT_MAX
            rk_data = st.rankdata(-preds, method='ordinal')

            t_score = preds[self.target_item] / preds[(rk_data == self.top_k).nonzero()][0]
        elif self.norm == 'rescale':
            preds[perm] = -StaticVars.FLOAT_MAX
            rk_data = st.rankdata(-preds, method='ordinal')

            max_val = rk_data[0]
            min_val = rk_data[-1]
            t_score = (max_val - preds[self.target_item]) / (max_val - min_val)
        else:  # default case
            tensor = F.softmax(torch.from_numpy(preds).float(), dim=0)
            preds = tensor.numpy()
            preds[perm] = -StaticVars.FLOAT_MAX

            t_score = preds[self.target_item]

        return self.alpha * t_score + (1 - self.alpha) * self.get_custom_score(np.sum(d))

    def add(self, d, s):
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if (mask_to_int not in self.visited_) and (self.budget > 0):
            t_score = self.get_score(d)
            hq.heappush(self.q, (int(not s), t_score, next(self.tiebraker), d.copy(), self.budget))

            self.visited_.add(mask_to_int)

# BruteForceSelection

In [11]:
class BruteForceSelection(BaseStrategy):
    class_name = 'BruteForce'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model):
        super().__init__(item, interactions, max_sequence_length, init_budget, model)

        self.q = Queue()
        self.q.enqueue(([False] * len(self.original_interactions), self.budget))

    def _expand_queue(self):
        m_mask = np.ma.getmask(self.ma_arr).copy()
        valid_items = np.where(np.logical_not(m_mask))[0]
        if valid_items.size > 1:
            for idx in valid_items:
                m_mask[idx] = not m_mask[idx]
                self.add(m_mask)
                m_mask[idx] = not m_mask[idx]

    def _next_item(self):
        mask, budget = self.q.dequeue()
        return (mask, budget)

    def next_comb(self, reverse=False):
        budget = self.budget

        if reverse: self.q.clear()

        if self.q.size() > 0:
            item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())
            self._expand_queue()
        else:
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr)
        return (seq, budget) if len(seq) else (None, budget)

    def add(self, d):
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if (mask_to_int not in self.visited_) and (self.budget > 0):
            self.budget -= 1
            self.q.enqueue((d.copy(), self.budget))
            self.visited_.add(mask_to_int)

# ComboSelection

In [12]:
class ComboSelection(BiDirectionalSelection):
    class_name = 'Combo'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model, weights=(1, 0), alpha=0.9, normalization='default'):
        super().__init__(item, interactions, max_sequence_length, init_budget, model, weights, alpha, normalization)

        self.alpha = 1

        self.q_init = Queue()
        self.q_init.enqueue((StaticVars.INT_MAX, [False] * len(self.original_interactions), self.budget))
        self.init_queue()

        self.tiebraker = itertools.count()
        self.q = []
        hq.heapify(self.q)

        self.alpha = alpha

    def init_queue(self):
        _, m_mask, budget = self.q_init.dequeue()
        m_mask = np.asarray(m_mask)

        valid_items = np.where(np.logical_not(m_mask))[0]
        for idx in valid_items:
            m_mask[idx] = not m_mask[idx]

            mask_to_int = int(''.join(map(str, m_mask.astype(int))), 2)
            if (mask_to_int not in self.visited_) and (self.budget > 0):
                t_score = self.get_score(m_mask)
                self.q_init.enqueue((t_score, m_mask.copy(), self.budget))

                self.visited_.add(mask_to_int)

            m_mask[idx] = not m_mask[idx]

        pair_combs = []
        for c in itertools.combinations(range(len(self.original_interactions)), 2):
            m = [False] * len(self.original_interactions)
            m[c[0]], m[c[1]] = not m[c[0]], not m[c[1]]
            pair_combs.append((self.q_init.get(c[0])[0] + self.q_init.get(c[1])[0], m.copy()))

        pair_combs.sort(key=operator.itemgetter(0))
        for c in pair_combs:
            self.budget -= 1
            self.q_init.enqueue((0, c[1], self.budget))

    def next_comb(self, reverse=False):
        budget = self.budget

        if self.q_init.size() > 0:
            s, item_mask, budget = self.q_init.dequeue()
            item_mask = np.asarray(item_mask)
            solved_flag = False
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())

            self.add(item_mask, False)
        elif self.q:
            solved_flag, item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())

            self._update_queue(solved_flag)
        else: self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr)
        return (seq, budget) if len(seq) else (None, budget)

# Get backend strategy

In [9]:
def get_backend_strategy(backend):
    if 'random' == backend:
        return RandomSelection
    elif 'most_sim' == backend:
        return MostSimilarSelection
    elif 'most_sim_jaccard' == backend:
        return MostSimilarSelectionByJaccard
    elif 'bfs' == backend:
        return LossSimilarSelection
    elif 'random_most_sim' == backend:
        return RandomMostSimilarSelection
    elif 'random_loss_sim' == backend:
        return RandomLossSimilarSelection
    elif 'fixed_loss_sim' == backend:
        return FixedRankingLossSimilarSelection
    elif 'dfs_loss_sim' == backend:
        return DFSwithLossSelection
    elif 'dfs_fixed_loss_sim' == backend:
        return DFSwithFixedRankingLossSelection
    elif 'bestFS_loss' == backend:
        return BestFSLossSelection
    elif 'bestFS_fixed_loss' == backend:
        return BestFSFixedLossSelection
    elif 'topdown_loss' == backend:
        return TopDownBestFSLossSelection
    elif 'bidirectional' == backend:
        return BiDirectionalSelection
    elif 'brute_force' == backend:
        return BruteForceSelection
    elif 'combo' == backend:
        return ComboSelection
    else: print('Unknown strategy')

# Run implemented strategies

In [10]:
# get currently working directory
base_dir = os.getcwd()

# load functions from other notebooks
helpers_file = os.path.join(base_dir, 'helpers.ipynb').replace("\\", "/")
%run $helpers_file

backend = 'random'
random_cfs = [

    # _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], negative_mode=True, sim_matrix=jaccard_sims_matrix, no_users= 10, init_budget=1000)
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11], negative_mode=True, sim_matrix=jaccard_sims_matrix, no_users= 10, init_budget=1000)
]

%store random_cfs


target position loop:   0%|          | 0/1 [00:00<?, ?it/s]

The backend used is: Random


users loop:   0%|          | 0/10 [00:00<?, ?it/s]

[-0.         -0.48867    -1.2704934  ...  1.442263    0.2114029
  0.10461938]
test2
[ 0.         1.0193417  1.1060333 ... -1.6374741 -1.2283658 -2.4511044]
[-0.        -1.0193417 -1.1060333 ...  1.6374741  1.2283658  2.4511044]
[1218  129  100 ... 3489 3214 3655]
-3.4028235e+38
1.8124814
1.8043554
11
1.8124814
test3
0.8401248
1.8124814
test2
[ 0.         1.1810582  1.1696018 ... -2.0357497 -1.1601081 -2.5264063]
[-0.        -1.1810582 -1.1696018 ...  2.0357497  1.1601081  2.5264063]
[1136  127  130 ... 3566 3049 3643]
-3.4028235e+38
1.9865843
1.9618725
11
1.9865843
test3
0.5999568
1.9865843
test2
[ 0.         1.5906975  0.672047  ... -2.3403883 -1.4878786 -1.8581343]
[-0.        -1.5906975 -0.672047  ...  2.3403883  1.4878786  1.8581343]
[1079   59  396 ... 3610 3264 3465]
-3.4028235e+38
2.1061394
2.090379
11
2.1061394
test3
0.3346629
2.1061394
test2
[ 0.          1.1000103   0.49245605 ... -1.1066442  -1.1231949
 -1.4803123 ]
[-0.         -1.1000103  -0.49245605 ...  1.1066442   1.123

target position loop: 10it [00:14,  1.42s/it]              

test2
[ 0.         1.2722664  0.5320109 ... -0.6029968 -1.7391344 -1.3537973]
[-0.        -1.2722664 -0.5320109 ...  0.6029968  1.7391344  1.3537973]
[1597   86  674 ... 2774 3604 3472]
-3.4028235e+38
1.7468643
1.7232453
11
1.7468643
test3
0.7358927
1.7468643
Stored 'random_cfs' (list)


In [15]:
backend = 'bfs'

bfs_yloss_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], negative_mode=True, sim_matrix=jaccard_sims_matrix, no_users= 10, init_budget=1000),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], negative_mode=True, sim_matrix=jaccard_sims_matrix, no_users= 10, init_budget=1000, early_term=True),
]

# %store bfs_yloss_cfs_positive
%store bfs_yloss_cfs

target position loop:   0%|          | 0/4 [00:00<?, ?it/s]

The backend used is: BFS


users loop:   0%|          | 0/500 [00:00<?, ?it/s]

target position loop: 20it [00:14,  1.39it/s]              

users loop:   0%|          | 0/500 [00:00<?, ?it/s]

...

users loop:   0%|          | 0/500 [00:00<?, ?it/s]

target position loop: 40it [00:43,  1.19s/it]

users loop:   0%|          | 0/500 [00:00<?, ?it/s]

target position loop: 40it [00:57,  1.43s/it]


Stored 'bfs_yloss_cfs' (list)


In [16]:
backend='bidirectional'

bidirectional_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], negative_mode=True, sim_matrix=jaccard_sims_matrix, no_users=10, init_budget=1000, alpha=1e-3, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], negative_mode=True, sim_matrix=jaccard_sims_matrix, no_users=10, init_budget=1000, alpha=0.5, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], negative_mode=True, sim_matrix=jaccard_sims_matrix, no_users=10, init_budget=1000, alpha=0.999, normalization='default'),
]

%store bidirectional_cfs

target position loop:   0%|          | 0/4 [00:00<?, ?it/s]

The backend used is: BiDirectional


users loop:   0%|          | 0/500 [00:00<?, ?it/s]

target position loop: 20it [12:48, 38.44s/it]              

users loop:   0%|          | 0/500 [00:00<?, ?it/s]

...

users loop:   0%|          | 0/500 [00:00<?, ?it/s]

target position loop: 40it [34:28, 55.84s/it]

users loop:   0%|          | 0/500 [00:00<?, ?it/s]

target position loop: 40it [45:16, 67.92s/it]


Stored 'bidirectional_cfs' (list)


In [17]:
backend='brute_force'

brute_force_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], negative_mode=True, sim_matrix=jaccard_sims_matrix, no_users=10, init_budget=100000),
]

%store brute_force_cfs

target position loop:   0%|          | 0/4 [00:00<?, ?it/s]

The backend used is: BruteForce


users loop:   0%|          | 0/500 [00:00<?, ?it/s]

target position loop: 20it [00:04,  4.70it/s]              

users loop:   0%|          | 0/500 [00:00<?, ?it/s]

...

users loop:   0%|          | 0/500 [00:00<?, ?it/s]

target position loop: 40it [00:12,  2.85it/s]

users loop:   0%|          | 0/500 [00:00<?, ?it/s]

target position loop: 40it [00:17,  2.32it/s]

Stored 'brute_force_cfs' (list)


In [14]:
backend='combo'
combo_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], negative_mode=True, sim_matrix=jaccard_sims_matrix, no_users=10, init_budget=1000, alpha=1e-3, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], negative_mode=True, sim_matrix=jaccard_sims_matrix, no_users=10, init_budget=1000, alpha=0.5, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], negative_mode=True, sim_matrix=jaccard_sims_matrix, no_users=10, init_budget=1000, alpha=0.999, normalization='default'),
]

%store combo_cfs

target position loop:   0%|          | 0/4 [00:00<?, ?it/s]

The backend used is: Combo


users loop:   0%|          | 0/500 [00:00<?, ?it/s]

target position loop: 20it [14:50, 44.52s/it]              

users loop:   0%|          | 0/500 [00:00<?, ?it/s]

...

users loop:   0%|          | 0/500 [00:00<?, ?it/s]

target position loop: 40it [33:50, 55.46s/it]

users loop:   0%|          | 0/500 [00:00<?, ?it/s]

target position loop: 40it [45:14, 67.87s/it]


Stored 'combo_cfs' (list)
